In [7]:
# # install neccesary plugins
# !pip install pandas
# !pip install selenium
# !pip install webdriver_manager
# !pip install bs4

In [1]:
# imports
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from datetime import datetime
from webdriver_manager.firefox import GeckoDriverManager

# update and run the latest Firefox webdriver
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Getting latest mozilla release info for v0.29.1


[WDM] - Trying to download new driver from https://github.com/mozilla/geckodriver/releases/download/v0.29.1/geckodriver-v0.29.1-win64.zip
[WDM] - Driver has been saved in cache [C:\Users\mn850\.wdm\drivers\geckodriver\win64\v0.29.1]


In [2]:
# dicationary to turn into final table
defi_dict = {}

# class ids
protocol = 'sc-caSCKo hKzJw'
token = 'sc-caSCKo dIpawx'
platform = 'sc-ksYbfQ jqDzNU css-4cffwv'
tvl = 'sc-ksYbfQ jqDzNU css-4cffwv'
html_id = 'sc-cHGsZl dBFEZZ css-1yh09yi'

# navigation xpaths
xpath_wait = "/html/body/div/div/div/div[2]/div/div/div[2]/div/div[1]/div[1]/div"
xpath_next_page = '/html/body/div/div/div/div[2]/div/div/div[2]/div/div[4]/div[3]/div'
xpath_pagenumber = '/html/body/div/div/div/div[2]/div/div/div[2]/div/div[4]/div[2]'

# sites to iterate through
site_dict = {'Protocols':'https://defillama.com/protocols',
             'Dexes':'https://defillama.com/protocols/dexes',
             'Assets':'https://defillama.com/protocols/assets',
             'Lending':'https://defillama.com/protocols/lending',
             'Yield':'https://defillama.com/protocols/yield',
             'Insurance':'https://defillama.com/protocols/insurance',
             'Options':'https://defillama.com/protocols/options',
             'Indexes':'https://defillama.com/protocols/indexes',
             'Staking':'https://defillama.com/protocols/staking'}


for item in site_dict.items():
    driver.get(item[1])
    
    # wait until the 'Name' element of the site has been loaded
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath_wait)))
    except:
        pass
    
    number_of_pages = int(driver.find_element_by_xpath(xpath_pagenumber).text.split()[-1:][0])
 
    # iterate through the pages
    for page in range(1,number_of_pages +1):

        # grab the html of each table row
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        html = soup.find_all("div", class_ = html_id)
        rows = html[0].find_all('div', attrs ={'style': 'height: 48px;'})

        # grab the elements from the different columns in the row
        for rownumber in range(len(rows)):
            defi_dict[rows[rownumber].find_all('div', attrs={'class': protocol})[0].text] = []
            #protocol
            try:
                defi_dict[rows[rownumber].find_all('div', attrs={'class': protocol})[0].text].append(item[0])
            except:
                defi_dict[rows[rownumber].find_all('div', attrs={'class': protocol})[0].text].append('Protocol not found')
            #token
            try:
                defi_dict[rows[rownumber].find_all('div', attrs={'class': protocol})[0].text].append(rows[rownumber].find_all('div', attrs={'class': token})[0].text)
            except:
                defi_dict[rows[rownumber].find_all('div', attrs={'class': protocol})[0].text].append('Token not found')
            #platform
            try:
                defi_dict[rows[rownumber].find_all('div', attrs={'class': protocol})[0].text].append(rows[rownumber].find_all('div', attrs={'class': platform})[0].text)
            except:
                defi_dict[rows[rownumber].find_all('div', attrs={'class': protocol})[0].text].append('Platform not found')
            #%change 1
            try:
                defi_dict[rows[rownumber].find_all('div', attrs={'class': protocol})[0].text].append(rows[rownumber].find_all('div')[11].text)
            except:
                defi_dict[rows[rownumber].find_all('div', attrs={'class': protocol})[0].text].append('1h Change not found')
            #%change 2                
            try:
                defi_dict[rows[rownumber].find_all('div', attrs={'class': protocol})[0].text].append(rows[rownumber].find_all('div')[13].text)
            except:
                defi_dict[rows[rownumber].find_all('div', attrs={'class': protocol})[0].text].append('1d Change not found')                
            #%change 3                
            try:
                defi_dict[rows[rownumber].find_all('div', attrs={'class': protocol})[0].text].append(rows[rownumber].find_all('div')[9].text)
            except:
                defi_dict[rows[rownumber].find_all('div', attrs={'class': protocol})[0].text].append('7d Change not found')
            #%change TVL                
            try:
                defi_dict[rows[rownumber].find_all('div', attrs={'class': protocol})[0].text].append(rows[rownumber].find_all('div', attrs={'class': tvl})[2].text)
            except:
                defi_dict[rows[rownumber].find_all('div', attrs={'class': protocol})[0].text].append('TVL not found')                 

        # go to the next page
        driver.find_element_by_xpath(xpath_next_page).click()


In [3]:
# convert dictionary into Pandas dataframe
df = pd.DataFrame.from_dict(defi_dict, 
                            orient='index',
                            columns = ['Category',
                                       'Symbol',
                                       'Chain',
                                       '1h Change',
                                       '1d Change',
                                       '7d Change',
                                       'TVL'])

# partially clean TVL text
df['TVL_converted'] = df['TVL'].str.replace('$','', regex = True).replace(',','', regex = True)

df['Snapshot_taken'] = datetime.now()

df = df.reset_index().rename(columns = {'index':'Protocol'})
df

,Protocol,Category,Symbol,Chain,1h Change,1d Change,7d Change,TVL,TVL_converted,Snapshot_taken
0,AAVE,Lending,AAVE,Ethereum,-16.51%,-13.21%,+4.93%,$11.3b,11.3b,2021-05-20 07:50:28.070004
1,MakerDAO,Protocols,MKR,Ethereum,-20.43%,-33.22%,+4.41%,$9.51b,9.51b,2021-05-20 07:50:28.070004
2,Polygon,Protocols,MATIC,Polygon,-12.93%,+39.16%,-1.45%,$8.98b,8.98b,2021-05-20 07:50:28.070004
3,Compound,Lending,COMP,Ethereum,-9.34%,-26.25%,+1.85%,$8.21b,8.21b,2021-05-20 07:50:28.070004
4,Curve,Dexes,CRV,Ethereum,-15.79%,-16.44%,+1.51%,$7.34b,7.34b,2021-05-20 07:50:28.070004
...,...,...,...,...,...,...,...,...,...,...
199,CoFiX,Dexes,COFI,Ethereum,-7.70%,-30.74%,-1.54%,"$222,439",222439,2021-05-20 07:50:28.070004
200,Shell Protocol,Dexes,-,Ethereum,+0.01%,+0.01%,0%,"$113,671",113671,2021-05-20 07:50:28.070004
201,SynLev,Protocols,SYN,Ethereum,-18.29%,-34.74%,+3.14%,"$93,174",93174,2021-05-20 07:50:28.070004
202,Robo-Advisor fo...,Yield,-,Ethereum,-6.49%,-98.81%,-0.20%,"$13,284",13284,2021-05-20 07:50:28.070004


In [4]:
# further clearn the TVL text
def value_to_float(x):
    if 'b' in x:
        return round(float(x.replace('b', '')) * 1_000_000_000,0)
    if 'm' in x:
        return round(float(x.replace('m', '')) * 1_000_000,0)
    else:
        return int(x)

df['TVL_converted'] = df['TVL_converted'].apply(value_to_float)
df['TVL_converted'] = df['TVL_converted'].astype('Int64')

In [5]:
df.to_csv('Defi llama stats ' + datetime.now().strftime('%Y.%m.%d - %Hh%Mmin') + '.csv',
          index = False)

In [6]:
import os
os.startfile('https://github.com/ovu-github/defi-llama-scraper/tree/main/data_extracts')